In [2]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [3]:
classes = ("Paper", "Paper pack", "Metal", "Glass", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
#            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/_boost_/mask_rcnn_swin-l.py')

root='../../dataset/'

epoch = 'best_bbox_mAP_epoch_35'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = [(512,512)] # Resize

# cfg.data.test.pipeline[1]['flip'] = True
# cfg.data.test.pipeline[1]['flip_direction'] = ['horizontal','vertical']
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/mask_rcnn_swin-l_5'

# cfg.model.roi_head.bbox_head.num_classes = 10
cfg.model.roi_head.bbox_head.num_classes = 8

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [4]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        # workers_per_gpu=cfg.data.workers_per_gpu,
        workers_per_gpu=3,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/mask_rcnn_swin-l_5/best_bbox_mAP_epoch_35.pth


In [6]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 13.5 task/s, elapsed: 362s, ETA:     0s

In [7]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

# class_num = 10
class_num = 8
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        n = j
        if n == 0:
            n = 1
        elif n == 1:
            n = 2
        elif n == 2:
            n = 3
        elif n == 3:
            n = 4
        elif n == 4:
            n = 6
        elif n == 5:
            n = 7
        elif n == 6:
            n = 8
        elif n == 7:
            n = 9
        for o in out[j]:
            prediction_string += str(n) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.9985531 576.84344 109.935326 757.88885 364...,test/0000.jpg
1,1 0.20836663 754.3574 647.5637 899.0122 1000.8...,test/0001.jpg
2,1 0.99924517 298.8783 300.61438 1024.0 749.885...,test/0002.jpg
3,9 0.9994867 156.65 273.41043 904.15454 817.132...,test/0003.jpg
4,1 0.9995521 188.82571 270.38376 876.9525 773.6...,test/0004.jpg
